In [5]:
import random

class Card:
    def __init__(self, suit, rank, value):
        self.suit = suit
        self.rank = rank
        self.value = value

    
    def __str__(self):

        return f'{self.rank} of {self.suit}'
    

class Deck:

    def __init__(self, num_decks=6):


        self.num_decks = num_decks

        self.cards = self.build_deck()

        self.plastic_card_position = None

        self.shuffle()



    def shuffle(self):

        random.shuffle(self.cards)

        self.plastic_card_position = random.randint(50, len(self.cards))

    def draw_card(self):

        if len(self.cards) == 0:

            self.shuffle()

        
        card = self.cards.pop()


        if len(self.cards) == self.plastic_card_position:
            self.shuffle()

        return card
    def build_deck(self):

        suits = ['Hearts', 'Diamonds', 'Clubs', 'Spades']

        ranks = ['2', '3', '4', '5', '6', '7', '8', '9', '10', 'J', 'Q', 'K', 'A']

        values = {'2': 2, '3':3, '4':4, '5':5, '6':6, '7':7, '8':8, '9': 9, 
                  '10': 10, 'J': 10, 'Q': 10, 'K': 10, 'A': 11}
        
        deck = []

        for p in range(self.num_decks):

            for suit in suits:

                for rank in ranks:
                    deck.append(Card(suit,rank,values[rank]))

        return deck    

    def remaining_cards(self):

        return len(self.cards)


In [21]:
class Hand:
    def __init__(self):
        self.cards = []
        self.value = 0
        self.is_soft = False
    def add_card(self, card):
        self.cards.append(card)
        self.calculate_value()

    def calculate_value(self):
        self.value = sum(card.value for card in self.cards)
        self.is_soft = any(card.rank == 'A' for card in self.cards)
        if self.is_soft and self.value > 21:
            self.value -= 10
    def is_blackjack(self):
        return self.value == 21 and len(self.cards) == 2
    def is_bust(self):
        return self.value > 21
    def show_hand(self):
        return ', '.join([str(card) for card in self.cards])
    

class Player:
    def __init__(self, chips=100):
        self.hand = Hand()
        self.chips = chips

    def receive_card(self, card):
        self.hand.add_card(card)

    def calculate_hand_value(self):
        return self.hand.value

    def take_action(self, deck):
        raise NotImplementedError("subclasses must implement")
    

class Dealer(Player):
    def action(self, deck):
        while self.hand.value < 17:

            self.receive_card(deck.draw_card())
    
    def show_hand(self):
        return f'dealer : {self.hand.show_hand()} | value; {self.calculate_hand_value()}'
    
class StrategyPlayer(Player): 
    def __init__(self, chips = 100, threshold = 0):

        super().__init__(chips)

        self.card_count = 0
        self.threshold = threshold

    def update_cardcount(self, card):
        
        if 2<= card.value <=6:
            self.card_count+= 1

        elif card.value >= 10 or card.rank == 'A':
            self.card_count -= 1

    def action(self, deck):
        while self.hand.value < 21 :
                    
            if self.card_count <= self.threshold:
                print(f"StrategyPlayer hits with card count {self.card_count}")
                self.receive_card(deck.draw_card())
                self.update_cardcount(self.hand.cards[-1])
            else:
                    print(f"StrategyPlayer stays with card count {self.card_count}")
                    break

    def show_hand(self):
        return f"StrategyPlayer's hand: {self.hand.show_hand()} | value: {self.calculate_hand_value()} | count {self.card_count}"
    
    


In [26]:
class Game:

    def __init__(self):

        self.players = []
        self.dealer = Dealer()

        self.deck = Deck()

        self.rounds_played = 0

    def dealinit(self):
        for player in self.players:
            player.receive_card(self.deck.draw_card())
            player.receive_card(self.deck.draw_card())
        self.dealer.receive_card(self.deck.draw_card())
        self.dealer.receive_card(self.deck.draw_card())
        

    def addplayer(self,player):

        self.players.append(player)

    def play_round(self):
        self.dealinit()

        for player in self.players:
            print(player.show_hand())
        
        print(self.dealer.show_hand())

        for player in self.players:
            player.action(self.deck)

        self.dealer.action(self.deck)

        for player in self.players:
            print(player.show_hand())

        print(self.dealer.show_hand())

        self.resolve_bets()


    def resolve_bets(self):
        dealer_value = self.dealer.calculate_hand_value()
        for player in self.players:
            player_value = player.calculate_hand_value()
            if player_value > 21 :
                print(f'player busts {player_value}')
            elif dealer_value > 21 or player_value > dealer_value:
                print(f'player wins {player_value}')

            else: print(f'dealer wins with {dealer_value}')

In [28]:

game = Game()


strategy_player = StrategyPlayer(threshold=0)
game.addplayer(strategy_player)

game.addplayer(Dealer()) 

for i in range(5):
    print(f"\n--- Round {i + 1} ---")
    game.play_round()
    print("\n")



--- Round 1 ---
StrategyPlayer's hand: 8 of Diamonds, 5 of Spades | value: 13 | count 0
dealer : 8 of Hearts, 10 of Spades | value; 18
dealer : 5 of Diamonds, K of Spades | value; 15
StrategyPlayer hits with card count 0
StrategyPlayer's hand: 8 of Diamonds, 5 of Spades, 9 of Hearts | value: 22 | count 0
dealer : 8 of Hearts, 10 of Spades | value; 18
dealer : 5 of Diamonds, K of Spades, A of Clubs, 3 of Diamonds | value; 19
player busts 22
dealer wins with 19



--- Round 2 ---
StrategyPlayer's hand: 8 of Diamonds, 5 of Spades, 9 of Hearts, 8 of Diamonds, Q of Diamonds | value: 40 | count 0
dealer : 8 of Hearts, 10 of Spades, A of Clubs, 10 of Diamonds | value; 29
dealer : 5 of Diamonds, K of Spades, A of Clubs, 3 of Diamonds, 7 of Diamonds, 6 of Hearts | value; 32
StrategyPlayer's hand: 8 of Diamonds, 5 of Spades, 9 of Hearts, 8 of Diamonds, Q of Diamonds | value: 40 | count 0
dealer : 8 of Hearts, 10 of Spades, A of Clubs, 10 of Diamonds | value; 29
dealer : 5 of Diamonds, K of Spad